In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [1]:
from gpt_converter import gptkeys
import os

os.environ["OPENAI_API_KEY"] = gptkeys.getone()

설치된 openai 의 버전을 체크합니다.


In [2]:
import openai

openai.__version__

'1.6.1'

## ChatOpenAI

OpenAI 사의 채팅 전용 Large Language Model(llm) 입니다.

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

`temperature`

- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`

- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`model_name`: 적용 가능한 모델 리스트

- 링크: https://platform.openai.com/docs/models


In [12]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 뭐야?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울입니다.'


## 프롬프트 템플릿의 활용


`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.
- 사용법: 리스트 형식으로 변수 이름을 정의합니다.


In [13]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

### LLMChain 객체


`LLMChain`

- LLMChain은 특정 PromptTemplate와 연결된 체인 객체를 생성합니다
- 사용법
  - `prompt`: 앞서 정의한 PromptTemplate 객체를 사용합니다.
  - `llm`: 언어 모델을 나타내며, 이 예시에서는 이미 어딘가에서 정의된 것으로 보입니다.


In [14]:
from langchain.chains import LLMChain

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [15]:
llm_chain.invoke({"country": "대한민국"})

{'country': '대한민국', 'text': '대한민국의 수도는 서울입니다.'}

### apply()

`apply()` 함수로 여러개의 입력에 대한 처리를 한 번에 수행할 수 있습니다.


In [16]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

llm_chain.apply(input_list)

[{'text': '호주의 수도는 캔버라입니다.'},
 {'text': '중국의 수도는 베이징(北京)입니다.'},
 {'text': '네덜란드의 수도는 암스테르담(Amsterdam)입니다.'}]

`text` 키 값으로 결과 뭉치가 반환되었음을 확인할 수 있습니다.

이를 반복문으로 출력한다면 다음과 같습니다.


In [17]:
# input_list 에 대한 결과 반환
result = llm_chain.apply(input_list)

# 반복문으로 결과 출력
for res in result:
    print(res["text"].strip())

호주의 수도는 캔버라입니다.
중국의 수도는 베이징(北京)입니다.
네덜란드의 수도는 암스테르담(Amsterdam)입니다.


### `generate()`


`generate()` 는 문자열 대신에 LLMResult를 반환하는 점을 제외하고는 apply와 유사합니다.

LLMResult는 토큰 사용량과 종료 이유와 같은 유용한 생성 정보를 자주 포함하고 있습니다.


In [18]:
# 입력값
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

# input_list 에 대한 결과 반환
generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='호주의 수도는 캔버라입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라입니다.'))], [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.'))], [ChatGeneration(text='네덜란드의 수도는 암스테르담(Amsterdam)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='네덜란드의 수도는 암스테르담(Amsterdam)입니다.'))]] llm_output={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 58, 'total_tokens': 115}, 'model_name': 'gpt-3.5-turbo'} run=[RunInfo(run_id=UUID('191c0b2a-fbd6-478b-a609-e134f6c19a9f')), RunInfo(run_id=UUID('b9b74da4-23c1-4b7b-a572-9c77ddce9264')), RunInfo(run_id=UUID('dec5419f-9044-49f5-aee5-820fe09063f0'))]


In [20]:
# 토큰 사용량 출력
generated_result.llm_output

{'token_usage': {'completion_tokens': 57,
  'prompt_tokens': 58,
  'total_tokens': 115},
 'model_name': 'gpt-3.5-turbo'}

### 2개 이상의 변수를 템플릿 안에 정의


2개 이상의 변수를 적용하여 템플릿을 생성할 수 있습니다.

이번에는 2개 이상의 변수(`input_variables`) 를 활용하여 템플릿 구성을 해보겠습니다.


In [23]:
# 질문 템플릿 형식 정의
template = "{area1} 와 {area2} 의 시차는 몇시간이야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='{area1} 와 {area2} 의 시차는 몇시간이야?')

In [24]:
# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [26]:
# 체인 실행: run()
print(llm_chain.invoke({"area1": "서울", "area2": "파리"}))

{'area1': '서울', 'area2': '파리', 'text': '서울과 파리의 시차는 8시간입니다. 서울이 파리보다 8시간 앞서 있습니다.'}


In [27]:
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())

파리와 뉴욕의 시차는 일반적으로 6시간입니다. 파리가 뉴욕보다 6시간 앞서 있습니다. 예를 들어, 파리가 오전 9시라면 뉴욕은 오전 3시입니다.
서울과 하와이의 시차는 서울이 하와이보다 19시간 빠릅니다. 예를 들어, 서울이 오전 9시라면 하와이는 전날 오후 2시입니다.
켄버라와 베이징의 시차는 2시간입니다. 켄버라는 오스트레일리아의 수도로 UTC+10 시간대에 위치하고, 베이징은 중국의 수도로 UTC+8 시간대에 위치합니다.


## stream: 실시간 출력

스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용합니다.

다음과 같이 `streaming=True` 로 설정하고 스트리밍으로 답변을 받기 위한 `StreamingStdOutCallbackHandler()` 을 콜백으로 지정합니다.


In [31]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [33]:
# 질의내용
question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

# 스트리밍으로 답변 출력
response = llm.invoke(question)

대한민국은 동아시아에 위치한 나라로, 한반도의 남쪽에 위치하고 있다. 수도는 서울이며, 인구는 약 5,100만 명으로 세계에서 27번째로 인구가 많은 나라이다. 대한민국은 민주공화국으로, 대통령이 국가의 수장이며 국회에서 법률을 제정한다.

대한민국은 경제적으로 선진화된 나라로, 세계에서 12번째로 큰 경제를 가지고 있다. 주요 산업은 제조업, 서비스업, IT 등이다. 대한민국은 세계에서 가장 빠른 인터넷 속도를 가지고 있으며, IT 기술과 전자제품 생산 분야에서 세계적으로 유명하다.

문화적으로는 한국의 전통문화와 현대 문화가 조화롭게 공존한다. 한국은 한글을 사용하는 유일한 나라로, 한글은 세계에서 가장 과학적이고 효율적인 문자체계로 알려져 있다. 또한 한국은 한류 열풍으로 유명하며, 한국의 음식, 음악, 드라마 등이 전 세계에서 사랑받고 있다.

대한민국은 자연환경도 다양하다. 산과 강, 바다 등이 풍부하며, 자연경관이 아름답다. 한라산, 설악산 등의 산은 등산객들에게 인기가 많고, 제주도는 관광지로 유명하다. 또한 한반도는 사계절의 변화가 뚜렷하며, 봄에는 벚꽃, 가을에는 단풍이 아름답다.

대한민국은 국제적으로도 중요한 위치를 가지고 있다. 북한과의 관계와 분단 문제, 동북아시아의 안보 문제 등이 주요 이슈이다. 또한 대한민국은 세계적으로 인정받는 문화, 스포츠, 영화 등의 이벤트와 축제를 개최하며 국제 사회에서 활발한 역할을 수행하고 있다.